# 1. Unzip ds2_dense.tar.gz

In [15]:
import tarfile
import os
import shutil

# 1. Set the path to the tar.gz archive and the directory to extract to
tar_path = './ds2_dense.tar.gz'
extract_path = './ds2_dense/'

# 2. Open and extract the tar.gz archive
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

# 3. Check if a nested directory (e.g., ds2_dense/ds2_dense) was created
nested = os.path.join(extract_path, 'ds2_dense')
if os.path.exists(nested):
    
    # Move all files from the nested folder up to the main extraction folder
    for item in os.listdir(nested):
        shutil.move(os.path.join(nested, item), extract_path)
    # Remove the now-empty nested directory
    shutil.rmtree(nested)

# 2. Load JSON annotation file of deepscore

In [16]:
import json
import pandas as pd

# 1. Load the training dataset JSON file
with open('./ds2_dense/deepscores_train.json') as f:
    data1 = json.load(f)

# 2. Load the test dataset JSON file
with open('./ds2_dense/deepscores_test.json') as f:
    data2 = json.load(f)

# 3. Extract training images metadata as a DataFrame
train_images = pd.DataFrame(data1['images'])

# 4. Extract training annotations and transpose to match image IDs
train_annots = pd.DataFrame(data1['annotations']).T

# 5. Extract test images metadata as a DataFrame
test_images = pd.DataFrame(data2['images'])

# 6. Extract test annotations and transpose to match image IDs
test_annots = pd.DataFrame(data2['annotations']).T


# 3. Splitting the Images into Train and Test sets

In [17]:
import os

# Define the base directories
image_dir = './ds2_dense/images'
train_dir = './ds2_dense/images/train'
test_dir = './ds2_dense/images/test'

# Create the training and testing image directory if they are not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [18]:
# Move train images to train directory
for image_filename in train_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(train_dir, image_filename)
   shutil.move(src_path, dest_path)

In [19]:
# Move train images to test directory
for image_filename in test_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(test_dir, image_filename)
   shutil.move(src_path, dest_path)

# 4. Generate Label Mapping (deepscores only)
The yolo model wants labels to be zero-based indexing


In [20]:
# 1. Access the label dictionary from the third element of the training data dictionary
label_dict = list(data1.values())[2]

# 2. Filter only the labels that belong to the 'deepscores' annotation set
#    (some labels in the JSON belong to MUSCIMA++)
filtered_items = [(k, v) for k, v in label_dict.items() if v['annotation_set'] == 'deepscores']

# 3. Enumerate the filtered items to assign a unique integer label to each category
#    Store the original ID, category name, and new label ID
cat_dict = {
    k: {
        'old_id': int(k),
        'name': v['name'],
        'label': i
    } for i, (k, v) in enumerate(filtered_items)
}

# 4. Convert the category dictionary to a DataFrame for easy viewing and analysis
df_labels = pd.DataFrame.from_dict(cat_dict, orient='index').reset_index(drop=True)
df_labels

,old_id,name,label
0,1,brace,0
1,2,ledgerLine,1
2,3,repeatDot,2
3,4,segno,3
4,5,coda,4
...,...,...,...
131,132,tuplet8,131
132,133,tuplet9,132
133,134,tupletBracket,133
134,135,staff,134


# 5. Attach Label to each obbox

In [21]:
# 1. Create a mapping from original category ID (as string) to new label ID
class_mapping = dict(zip(df_labels['old_id'].astype(str), df_labels['label']))

# 2. Define a function to map a list of category IDs to a single label
#    It takes the maximum valid mapped label from the list (if any)
def map_label(cat_ids):
    return max([class_mapping.get(str(cid)) for cid in cat_ids if str(cid) in class_mapping], default=None)

# 3. Apply the label mapping function 
train_annots['label'] = train_annots['cat_id'].apply(map_label)
test_annots['label'] = test_annots['cat_id'].apply(map_label)

train_annots.head(10)


,a_bbox,o_bbox,cat_id,area,img_id,comments,label
1020,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134
1021,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134
1022,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1
1023,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1
1024,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1
1025,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1
1026,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1
1027,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1
1028,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1
1029,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1


# 6. Adjust bounding box for 0 valued w/h

In [22]:
# 1. Define a function to fix zero-size bounding boxes
#    If the width or height of the box is zero, slightly expand it by 1 pixel in both directions
def adjust_bbox(bbox):
    x_min, y_min, x_max, y_max = bbox
    if x_min == x_max:
        x_min -= 1
        x_max += 1
    if y_min == y_max:
        y_min -= 1
        y_max += 1
    return [x_min, y_min, x_max, y_max]

# 2. Apply the bounding box adjustment function 
train_annots['bbox'] = train_annots['a_bbox'].apply(adjust_bbox)
test_annots['bbox'] = test_annots['a_bbox'].apply(adjust_bbox)

train_images.head(10)


,id,filename,width,height,ann_ids
0,2,lg-94161796-aug-gonville--page-3.png,1960,2772,"[1113019, 1113020, 1113021, 1113022, 1113023, ..."
1,3,lg-139957803-aug-emmentaler--page-5.png,1960,2772,"[182814, 182815, 182816, 182817, 182818, 18281..."
2,4,lg-63506682-aug-emmentaler--page-6.png,1960,2772,"[435136, 435137, 435138, 435139, 435140, 43514..."
3,7,lg-102548668-aug-gonville--page-4.png,1960,2772,"[998808, 998809, 998810, 998811, 998812, 99881..."
4,8,lg-233786100286899765-aug-gutenberg1939--page-...,2970,4201,"[774295, 774296, 774297, 774298, 774299, 77430..."
5,9,lg-26406557-aug-lilyjazz--page-1.png,1960,2772,"[473724, 473725, 473726, 473727, 473728, 47372..."
6,10,lg-695667819306673755-aug-lilyjazz-.png,1960,2772,"[1032754, 1032755, 1032756, 1032757, 1032758, ..."
7,11,lg-156284447442202986-aug-gutenberg1939--page-...,1960,2772,"[53456, 53457, 53458, 53459, 53460, 53461, 534..."
8,12,lg-101766503886095953-aug-gonville--page-4.png,1960,2772,"[278463, 278464, 278465, 278466, 278467, 27846..."
9,13,lg-69678954220027474-aug-beethoven--page-176.png,1960,2772,"[219162, 219163, 219164, 219165, 219166, 21916..."


# 7. Merge Annotation and Image df

In [23]:
# 1. Rename the 'id' column in train_images to 'img_id' for consistency
train_images.rename(columns={'id': 'img_id'}, inplace=True)
test_images.rename(columns={'id': 'img_id'}, inplace=True)

# 2. Convert 'img_id' in annots to string type to ensure proper merging
train_annots['img_id'] = train_annots['img_id'].astype(str)
test_annots['img_id'] = test_annots['img_id'].astype(str)

# 3. Convert 'img_id' in images to string type to match the annotation DataFrame
train_images['img_id'] = train_images['img_id'].astype(str)
test_images['img_id'] = test_images['img_id'].astype(str)

# 4. Merge annotations with image metadata on 'img_id'
train_df = pd.merge(train_annots, train_images, on='img_id')
test_df = pd.merge(test_annots, test_images, on='img_id')

train_df.head(10)


,a_bbox,o_bbox,cat_id,area,img_id,comments,label,bbox,filename,width,height,ann_ids
0,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134,"[116.0, 139.0, 2315.0, 206.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
1,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134,"[116.0, 309.0, 2315.0, 376.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
2,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1,"[1880.0, 561.0, 1911.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
3,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1,"[1883.0, 578.0, 1911.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
4,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1,"[1827.0, 561.0, 1857.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
5,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1,"[1827.0, 578.0, 1857.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
6,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1,"[1773.0, 561.0, 1804.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
7,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1,"[1773.0, 578.0, 1804.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
8,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1,"[1718.0, 561.0, 1748.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
9,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1,"[1718.0, 578.0, 1748.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."


# 8. Convert BBOX to YOLO format

In [24]:
# 1. Define a function to convert bounding box coordinates to YOLO format
#    The YOLO format is: [x_center, y_center, width, height] — all normalized to [0, 1]
def convert_to_yolo(bbox, width, height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / width
    y_center = (y_min + y_max) / 2 / height
    w = (x_max - x_min) / width
    h = (y_max - y_min) / height
    return [x_center, y_center, w, h]

# 2. Apply the YOLO conversion to each row in the training set
train_df['yolo_bbox'] = train_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)
test_df['yolo_bbox'] = test_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)

train_df.head(10)


,a_bbox,o_bbox,cat_id,area,img_id,comments,label,bbox,filename,width,height,ann_ids,yolo_bbox
0,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134,"[116.0, 139.0, 2315.0, 206.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.5, 0.05015993021227101, 0.904566022213081, ..."
1,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134,"[116.0, 309.0, 2315.0, 376.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.5, 0.09959290491421925, 0.904566022213081, ..."
2,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1,"[1880.0, 561.0, 1911.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7797202797202797, 0.16356498982262285, 0.01..."
3,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1,"[1883.0, 578.0, 1911.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7803373097490744, 0.16836289619075312, 0.01..."
4,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1,"[1827.0, 561.0, 1857.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7577128753599341, 0.16356498982262285, 0.01..."
5,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1,"[1827.0, 578.0, 1857.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7577128753599341, 0.16836289619075312, 0.01..."
6,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1,"[1773.0, 561.0, 1804.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7357054709995886, 0.16356498982262285, 0.01..."
7,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1,"[1773.0, 578.0, 1804.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7357054709995886, 0.16836289619075312, 0.01..."
8,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1,"[1718.0, 561.0, 1748.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7128753599341835, 0.16356498982262285, 0.01..."
9,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1,"[1718.0, 578.0, 1748.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7128753599341835, 0.16836289619075312, 0.01..."


# 9. Output YOLO training information as txt

In [25]:
import os


def export_yolo_txt(df, output_dir):
    # 1. Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # 2. Group the DataFrame by image filename
    for fname, group in df.groupby('filename'):
        # Construct the .txt file path by replacing the image extension
        txt_path = os.path.join(output_dir, os.path.splitext(fname)[0] + '.txt')
        
        # 3. Open the .txt file and write annotations line by line
        with open(txt_path, 'w') as f:
            for _, row in group.iterrows():
                cls = row['label']
                x, y, w, h = row['yolo_bbox']
                f.write(f"{cls} {x} {y} {w} {h}\n")

# Export YOLO annotations
export_yolo_txt(train_df, './ds2_dense/labels/train')
export_yolo_txt(test_df, './ds2_dense/labels/test')


# 10. Prepare pairwise ds2 for pairwised yolo training and testing

In [26]:
import os
import numpy as np
from PIL import Image
from scipy.ndimage import convolve

# 1. Define parameters for staff line detection
gray_threshold = 250
kernel_width = 100
response_thresh = 10
min_black_ratio = 0.5
min_gap_between_lines = 4

# 2. Detect horizontal staff lines in the input grayscale image
def detect_staff_lines_np(img_np):
    binary = (img_np < gray_threshold).astype(np.uint8)
    kernel = np.ones((1, kernel_width), dtype=np.uint8)
    response = convolve(binary, kernel)
    candidates = (response > response_thresh).astype(np.uint8)
    image_width = img_np.shape[1]
    min_black = int(min_black_ratio * image_width)

    valid_y = [
        y for y in range(binary.shape[0])
        if np.max(candidates[y]) > 0 and np.sum(binary[y]) >= min_black
    ]

    # 3. Remove nearby duplicate lines using a minimum gap threshold
    def deduplicate_lines(y_coords, min_gap=4):
        y_coords = sorted(y_coords)
        deduped = []
        for y in y_coords:
            if not deduped or abs(y - deduped[-1]) >= min_gap:
                deduped.append(y)
        return deduped

    return deduplicate_lines(valid_y, min_gap_between_lines)

# 4. Group every 5 staff lines into a complete staff system
def group_staff_lines(y_coords):
    y_coords = sorted(y_coords)
    groups = []
    group = []
    for y in y_coords:
        group.append(y)
        if len(group) == 5:
            groups.append(group)
            group = []
    return groups

# 5. Crop image based on grouped staff lines and adjust corresponding YOLO labels
def crop_and_adjust_labels(image_path, label_path, image_output_dir, label_output_dir):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    img = Image.open(image_path).convert("RGB")
    gray = img.convert("L")
    np_gray = np.array(gray)
    staff_lines = detect_staff_lines_np(np_gray)
    staff_groups = group_staff_lines(staff_lines)

    # 6. Load YOLO labels from the label file (if available)
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    cls, xc, yc, w, h = map(float, parts)
                    labels.append((cls, xc, yc, w, h))

    # 7. For each pair of staff systems, crop and adjust labels accordingly
    for i in range(0, len(staff_groups), 2):
        groups_in_pair = staff_groups[i:i+2]
        if not groups_in_pair:
            continue

        all_ys = [y for group in groups_in_pair for y in group]
        margin = int(0.25 * (max(all_ys) - min(all_ys)))  # Add 25% vertical margin
        y_min = max(0, min(all_ys) - margin)
        y_max = min(img.height, max(all_ys) + margin)

        # 8. Crop the image based on the selected vertical region
        cropped = img.crop((0, y_min, img.width, y_max))
        pair_img_name = f"{base_name}_pair{i//2}.jpg"
        pair_lbl_name = f"{base_name}_pair{i//2}.txt"
        cropped.save(os.path.join(image_output_dir, pair_img_name))

        # 9. Adjust YOLO bounding boxes to fit the cropped image
        new_lines = []
        for cls, xc, yc, w, h in labels:
            abs_yc = yc * img.height
            abs_h = h * img.height
            box_top = abs_yc - abs_h / 2
            box_bottom = abs_yc + abs_h / 2

            if box_bottom < y_min or box_top > y_max:
                continue

            new_yc = (abs_yc - y_min) / (y_max - y_min)
            new_h = abs_h / (y_max - y_min)

            # 10. Skip any labels that fall outside the normalized range
            if new_yc < 0 or new_yc > 1 or new_h <= 0 or new_h > 1:
                continue

            new_line = f"{int(cls)} {xc:.6f} {new_yc:.6f} {w:.6f} {new_h:.6f}"
            new_lines.append(new_line)

        # 11. Save the adjusted YOLO labels to a .txt file
        with open(os.path.join(label_output_dir, pair_lbl_name), "w") as f:
            f.write("\n".join(new_lines))


In [27]:
# Define directory of pairwise
original_image_dir = "./ds2_dense/images/test"
original_label_dir = "./ds2_dense/labels/test"
pairwise_image_dir = "./ds2_pairwise/images/test"
pairwise_label_dir = "./ds2_pairwise/labels/test"
os.makedirs(pairwise_image_dir, exist_ok=True)
os.makedirs(pairwise_label_dir, exist_ok=True)

image_files = [f for f in os.listdir(original_image_dir) if f.endswith(('.jpg', '.png'))]
for file in image_files:
    crop_and_adjust_labels(
        os.path.join(original_image_dir, file),
        os.path.join(original_label_dir, os.path.splitext(file)[0] + ".txt"),
        pairwise_image_dir,
        pairwise_label_dir
    )

print("pairwised finished, results are available at:")
print(f"images: {pairwise_image_dir}")
print(f"labels: {pairwise_label_dir}")

pairwised finished, results are available at:
images: ./ds2_pairwise/images/test
labels: ./ds2_pairwise/labels/test


# 11. Save Yolo Training Meta information

In [28]:
#Write deepscore dense yaml
import yaml

yaml_dict = {
    'path': os.path.abspath('./ds2_dense'),
    'train': 'images/train',
    'val': 'images/test',
    'names': df_labels.set_index('label')['name'].to_dict()
}

with open('deepscores.yaml', 'w') as f:
    yaml.dump(yaml_dict, f)


In [29]:
# Write deepscore pairwise yaml
yaml_dict = {
    'path': os.path.abspath('./ds2_pairwise'),
    'train': 'images/train',
    'val': 'images/test',
    'names': df_labels.set_index('label')['name'].to_dict()
}

with open('deepscores_pairwise.yaml', 'w') as f:
    yaml.dump(yaml_dict, f)


# 12. Uncomment below code to Train Yolo Model or run yolo_train.py

In [30]:
# from ultralytics import YOLO
# import torch
# model = YOLO('yolov8m.pt') 

# # Check if CUDA is available and configure training accordingly

# device = torch.device("cuda:0")

# if torch.cuda.is_available():
#     print('Training with GPU.')
#     results = model.train(data="deepscores.yaml", epochs=10, batch=3, device=device, patience=0, deterministic=True, amp=True)
    
# else:
#     results = model.train(data="deepscores.yaml", imgsz=640, rect=True, epochs = 10, batch=4)
